<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/zlaczenie-istotnych-kolumn/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [15]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-07 17:14:52--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz.1’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==> SIZE spark-3.3.1-bin-hadoop2.tgz ... 274099817
==> PASV ... done.    ==> RETR spark-3.3.1-bin-hadoop2.tgz ... done.
Length: 274099817 (261M) (unauthoritative)

spark-3.3.1-bin-had 100%[===================>] 261.40M  12.2MB/s    in 18s     

2022-11-07 17:15:12 (14.8 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz.1’ saved [274099817]



In [16]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [17]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [18]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-07 17:15:17--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.210.25, 13.226.210.114, 13.226.210.117, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236041161 (225M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.1’

name.basics.tsv.gz. 100%[===================>] 225.11M  32.7MB/s    in 6.5s    

2022-11-07 17:15:23 (34.8 MB/s) - ‘name.basics.tsv.gz.1’ saved [236041161/236041161]

--2022-11-07 17:15:23--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.210.25, 13.226.210.114, 13.226.210.117, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286770276 (273M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.1’

title.akas.tsv.gz.1 100%[==================

##Rozpakowanie danych

In [19]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [20]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [21]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00723...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00544...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [22]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [25]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (12060121, 5)
df_title_principals_selected dataframe size:  (52924405, 4)
df_title_basic_selected dataframe size:  (9354673, 5)
joined dataframe size:  (60141511, 8)
joined dataframe size:  (60141511, 12)


## Wybierzmy z tabeli tylko aktorów i aktorki

In [26]:
data = data.filter((data.category == "actor") | (data.category == "actress"))

# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [39]:
sample = data.rdd.takeSample(False, 5)

In [40]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,tconst,nconst,primaryName,birthYear,deathYear,knownForTitles,category,characters,titleType,originalTitle,isAdult,genres
0,tt4954874,nm0245164,Nicolas Duvauchelle,1980,\N,"tt0481349,tt0795370,tt1661420,tt0856288",actor,"[""Pierre Viviani""]",tvEpisode,Episode #1.5,0,"Crime,Drama,Mystery"
1,tt3678530,nm0149218,Chafûrin,1961,\N,"tt0131179,tt0139774,tt0250921,tt0138919",actor,"[""Inspector Juzo Megure""]",tvEpisode,Kogoro in the Bar: Part 2,0,"Action,Animation,Comedy"
2,tt0521319,nm0250436,Vince Edwards,1928,1996,"tt0051959,tt0049406,tt0062886,tt0054519",actor,"[""Dr. Ben Casey""]",tvEpisode,"Pull the Wool Over Your Eyes, Here Comes the C...",0,Drama
3,tt10412008,nm0235022,Diana Douglas,1923,2015,"tt0083470,tt0093748,tt0311110,tt0044212",actress,"[""Lily Chernak Donnelly (1970-1973)""]",tvEpisode,Episode #1.719,0,Drama
4,tt4745214,nm1310034,Serge Gisquière,\N,\N,"tt1328968,tt1010363,tt1576421,tt1738562",actor,"[""Peter Watson""]",tvEpisode,Lendemain de noces,0,Romance
